# DVA263 project

# Libraries

Declaring all libraries in use

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
import os
import cnn as cnn
import preprocessing as pre
import visualization as vis


# Load Dataset

In [2]:
def load_cnn():
    """Load the datasets for the CNN input"""
    CNN_train = "../dataset/CNN_Input/train/"
    CNN_test = "../dataset/CNN_Input/test/"
    CNN_train = pre.import_images(CNN_train, (150,150),batch_size=16, dataset=True)
    CNN_test = pre.import_images(CNN_test, (150,150), batch_size=16,  dataset=True)
    return CNN_train, CNN_test

# Prepare Data

In [3]:
def prep_cnn(cnn_train, cnn_test):
    """Add noise images to train and test set and normalize"""
    g_noisy_train_data = cnn_train.map(pre.add_gaussian_noise)
    snp_noisy_train_data = cnn_train.map(pre.add_salt_and_pepper_noise)
    combined_train_data = cnn_train.concatenate(g_noisy_train_data).concatenate(snp_noisy_train_data)

    percentage_noisy_images = 0.2
    total_test_images = len(CNN_test) * 16  # Total images in test data
    num_noisy_images = int(total_test_images * percentage_noisy_images)

    g_noisy_test_data = cnn_test.take(num_noisy_images).map(pre.add_gaussian_noise)
    snp_noisy_test_data = cnn_test.take(num_noisy_images).map(pre.add_salt_and_pepper_noise)
    remaining_test_data = cnn_test.skip(num_noisy_images)

    combined_test_data = g_noisy_test_data.concatenate(remaining_test_data)
    combined_test_data = combined_test_data.concatenate(snp_noisy_test_data)

    test_data_norm, train_data_norm = pre.normalize_dataset(combined_train_data, combined_test_data)
    return train_data_norm, test_data_norm


# Build and Compile Model

In [4]:
def build_cnn():
    model = cnn.build_cnn_model()
    return model

#  Train Model

In [5]:
def train_cnn(model,cnn_train, cnn_test):
    history = cnn.train_model(model, cnn_train, cnn_test)
    return history

# Putting everything together

Main section putting each part together and running the classification task

In [6]:
CNN_train, CNN_test = load_cnn()
CNN_train, CNN_test = prep_cnn(CNN_train, CNN_test)
CNN_model = build_cnn()
CNN_history = train_cnn(CNN_model, CNN_train, CNN_test)

CNN_model.predict(CNN_test) # here we will use the output from the autoencoder.

Found 83 files belonging to 4 classes.
Found 62 files belonging to 4 classes.
Epoch 1/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.2839 - loss: 1.4051 - val_accuracy: 0.2500 - val_loss: 1.3829 - learning_rate: 0.0010
Epoch 2/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.2888 - loss: 1.3826 - val_accuracy: 0.2581 - val_loss: 1.3806 - learning_rate: 0.0010
Epoch 3/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.2976 - loss: 1.3813 - val_accuracy: 0.3065 - val_loss: 1.3752 - learning_rate: 0.0010
Epoch 4/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.3149 - loss: 1.3701 - val_accuracy: 0.3065 - val_loss: 1.3651 - learning_rate: 0.0010
Epoch 5/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.4276 - loss: 1.3658 - val_accuracy: 0.3065 - val_loss: 1.3705 - learning_rate: 0.0010
Epoch 6/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.4763 - loss: 1.3532 - val_accuracy: 0.3306 - val_loss: 1.3321 - learning_rate: 0.0010
Epoch 7/70
18/18

array([[4.01501689e-04, 9.99405026e-01, 9.39732203e-08, 1.93349493e-04],
       [9.97116804e-01, 2.79292115e-03, 9.00011073e-05, 2.98453386e-07],
       [4.86821216e-03, 9.82760429e-01, 1.06671200e-06, 1.23703377e-02],
       [1.04856406e-06, 9.32216644e-04, 3.46913039e-05, 9.99032021e-01],
       [2.81064558e-07, 4.11402361e-05, 1.95980276e-04, 9.99762595e-01],
       [1.15318551e-04, 6.90725225e-04, 3.49751674e-03, 9.95696425e-01],
       [1.37567826e-04, 1.35134425e-04, 1.47215538e-02, 9.85005677e-01],
       [2.46211403e-05, 9.99971390e-01, 7.01422778e-08, 3.87693945e-06],
       [2.61141686e-04, 9.99225855e-01, 7.06311198e-07, 5.12357918e-04],
       [3.58725742e-08, 1.48907304e-04, 3.54350595e-05, 9.99815643e-01],
       [2.93638072e-06, 1.18213084e-05, 9.99938846e-01, 4.62965545e-05],
       [9.98542309e-01, 1.45719643e-03, 5.06862932e-07, 4.81535558e-08],
       [1.38337628e-05, 9.47452863e-05, 2.89297383e-03, 9.96998549e-01],
       [9.98419046e-01, 1.57371047e-03, 4.76206097e